We have to solve the following problem

$$\begin{cases}
\partial_tu(t,x)=\beta \partial_x^2 u(t,x)+f(t,x)\\
u(t=0, x)=u^0\\
u(t=0,x)=u_1 cos(\frac{\pi x}{2L}) \\
\partial_x(t,x=L)=0  \\
\end{cases} (0)$$
with $f(t,x)=(T_0 + \frac{x^2}{L^2}(T_1-T_0))\frac{1}{2-e^{\frac{-t}{\tau}}}$ and $T_1>0$

Let us remind the problems each team had to solve

**Team 1's problem**
$$ \begin{cases}
\partial_t u = f\\
u(t=0,x)=u^0=u_1 cos(\frac{\pi x}{2L}) \\
\end{cases} (1)$$
Leading to the following discretized problem 
$\begin{cases} 
v^0=u^0 \\
\forall n \in \llbracket 1,N \rrbracket, v^{n+1}=v^n+h_tf(t^n, v^n)
\end{cases}$

As $(v_n)_{n \in \N}\in (\R^{J+1})^{\N}$, we can rewrite the problem distinguishing each coordinate of $v_n$. We denote 
$$f^n :\begin{cases} \R^{J+1} \to \R^{J+1}\\
X= \begin{bmatrix}x_1 \\
\vdots \\
x_{J+1} \\
\end{bmatrix} \mapsto \begin{bmatrix} f(t^n,x_1)\\
\vdots \\
f(t^n,x_{J+1}) \end{bmatrix}\\
\end{cases}$$ 

yielding to the following discretized problem in matrix form: 

$\begin{cases} 
V^0=U^0=\begin{bmatrix} u_1 cos(\frac{\pi x_1}{2L}) \\
\vdots \\
u_1 cos(\frac{\pi x_{J+1}}{2L})\end{bmatrix} \\
\forall n \in \llbracket 1,N \rrbracket, V^{n+1}=V^n+h_t F^n(V^n) \\
\end{cases}$

**Team 3's problem**
$$ \begin{cases}
f =+ \beta \partial^2_x u =0\\
u(t=0,x)=u_1 cos(\frac{\pi x}{2L}) \\
\partial_x(t,x=L)=0
\end{cases} (3)$$
Leading to the following discretized problem when $\alpha=0$
$\begin{cases} 
V^0=U^0 \\
\forall n \in \llbracket 1,N \rrbracket, A V^{n}=F^{n}X
\end{cases}$

with $$ X = \begin{bmatrix} 
 x_1 \\
\vdots \\
x_{J+1} \\
\end{bmatrix} $$

$$A = -\frac{\beta}{h_x^2}\begin{bmatrix} 
 2 & -1 &  & &  \\
 -1 & 2 & -1 & & \\
  & \ddots & \ddots & \ddots & \\
  & & -1 & 2 & -1 \\
  & & & -2 & 2 \\
\end{bmatrix} 
$$

**Team 5's problem**
$$ \begin{cases}
\partial_t u = \beta \partial^2_x u \\
u(t, x=0)=a \\
u(t=0,x)=u_1 cos(\frac{\pi x}{2L}) \\
\partial_x(t,x=L)=0
\end{cases} (5) $$
Leading to the following discretized problem 
$\begin{cases} 
V^0=U^0 \\
\forall n \in \llbracket 1,N \rrbracket, V^{n+1}=(\frac{-h_t \beta}{h_x^2}D + I_{J+1})V^n +C = BV^n + C
\end{cases}$

with 
$$ V^n = \begin{bmatrix} 
 v_1^n \\
 \vdots \\
v_{J+1}^n \\
\end{bmatrix} \\~\\

D= \begin{bmatrix}
 2 & -1 &  & & 0 \\
 -1 & 2 & -1 &  &  \\
  & \ddots & \ddots & \ddots &  \\
 &  & -1 & 2 & -1\\
0 &  &  & -1 & 1\\ \end{bmatrix} \\~\\
C= \begin{bmatrix}
\frac{\beta a}{h_x^2} \\
0 \\
\vdots \\
0 \end{bmatrix}$$

We noticed that $(0) \iff (1)+(3)+(5)$ after simplification of the factor 2 appearing when summing the problems

Thus we thought of summing up all of our discretized problems as well to get the expression of our approximated solution solving (0). 

This gave us

$\begin{cases} 
V^0=U^0 \\
\forall n \in \llbracket 1,N \rrbracket, V^{n+1}= \frac{1}{2}[ (I_{J+1} + B - A) V^n +h_t F^n(V^n)+ F^{n}(X) + C]
\end{cases}$

We can thus implement this new numerical scheme in python to find our approximate solution

In [1]:
import numpy as np

#Parameters definiton 
L= 10
J= 100
N=100
T=10
u1=1
T0=1
T1=1
tau=1
k=1

# Source
def f(t,x):
    return T0*(1+np.cos(k*np.pi*x/L))*1/(1+np.exp(-t/tau))

#Def of the matrices of the numerical scheme
def matrices_def(L,J,T,N, beta):
    h_t=T/(N+1)
    h_x=L/(N+1)
    
    D= 2*np.eye(J+1)-1*np.eye(J+1, k=-1)-1*np.eye(J+1, k=1)
    D[J][J]-=1
    B = -h_t*beta/h_x**2*D+np.eye(J+1)
    
    A1=2*np.eye(J+1)-1*np.eye(J+1, k=-1)-1*np.eye(J+1, k=1)
    A1[J][J-1]-=1
    A=-beta/h_x**2*A1
    return A,B

#Numerical scheme 
def num_scheme_135(f,u1, J, L, N, T, a, beta, U0): 
    
    def F(X,n):
        return np.array([T0*(1+np.cos(k*np.pi*x/L))*1/(1+np.exp(-n*h_t/tau)) for x in X])
    
    h_x=L/(J+1)
    h_t=T/(N+1)
    X = np.array([j*h_x for j in range(1,J+2)])
    I = np.eye(J+1)
    A, B = matrices_def(L,J,T,N, beta)
    V0=np.array([u1*np.cos((np.pi*j*h_x)/(2*L)) for j in range(1,J+2)])
    V=[V0]
    Time=[0]
    
    for n in range(1, N+1):
        Vn=V[-1]
        Vnext=1/2*np.dot(I+B-A,Vn)+h_t*F(Vn,n)+F(X,n)
        V.append(Vnext)
        Time.append(n*h_t)
        
    return V, Time, X 